In [1]:
import os
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-01-16 20:04:27.148146: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-16 20:04:27.548778: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-16 20:04:27.548844: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-16 20:04:27.563581: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has alre

In [2]:
# Load data from a single file
def load_data(data_path):
    df = pd.read_csv(data_path)
    data = df['filename'].tolist()
    labels = df['label'].tolist()
    return data, labels

In [3]:
# Preprocess audio data
def preprocess_audio(file_path, target_duration=10):
    audio, _ = librosa.load(file_path, sr=None)

    target_length = int(target_duration * _)
    if len(audio) < target_length:
        audio = np.pad(audio, (0, target_length - len(audio)))
    else:
        audio = audio[:target_length]

    mfccs = librosa.feature.mfcc(y=audio, sr=_, n_mfcc=13)
    return mfccs

In [4]:
# Model architecture
def build_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [5]:
# Train the model
def train_model(X_train, y_train, X_val, y_val):
    model = build_model(input_shape=X_train.shape[1:])
    optimizer = Adam(learning_rate=0.001)  # Adjust the learning rate
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Data Augmentation
    datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1, zoom_range=0.1, horizontal_flip=True)
    datagen.fit(X_train)

    # Early Stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Model Training
    model.fit(datagen.flow(X_train, y_train, batch_size=32), validation_data=(X_val, y_val), epochs=20, callbacks=[early_stopping], verbose=1)

    # Save the trained model to a file
    model.save('new_model.h5')

    return model


In [8]:
# Main function
def main():

    train_data_path = 'Hindi_train.csv'
    test_data_path = 'Hindi_test.csv'

    # Load training data
    data_train, labels_train = load_data(train_data_path)

    # Load testing data
    data_test, labels_test = load_data(test_data_path)

    # Convert string labels to integers
    y_train = np.array([1 if label == 'Yes' else 0 for label in labels_train])
    y_test = np.array([1 if label == 'Yes' else 0 for label in labels_test])

    # Combine datasets for k-fold cross-validation
    data_combined = data_train + data_test
    labels_combined = y_train.tolist() + y_test.tolist()

    # Convert to numpy arrays
    X_combined = np.array([preprocess_audio(os.path.join('./audios/', filename)) for filename in tqdm(data_combined, desc="Processing Audio")])
    y_combined = np.array(labels_combined)

    # Initialize cross-validator
    n_splits = 5  # You can adjust the number of folds
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    fold = 1
    accuracies = []
    all_true_labels = []
    all_pred_labels = []

    for train_index, val_index in skf.split(X_combined, y_combined):
        X_train, X_val = X_combined[train_index], X_combined[val_index]
        y_train, y_val = y_combined[train_index], y_combined[val_index]

        X_train = X_train[..., np.newaxis]
        X_val = X_val[..., np.newaxis]

        # Train the model
        model = train_model(X_train, y_train, X_val, y_val)

        # Predict on the validation set
        y_pred_probs = model.predict(X_val)
        y_pred = (y_pred_probs > 0.5).astype(int)

        # Evaluate accuracy
        accuracy = accuracy_score(y_val, y_pred)
        print(f"Fold {fold} - Validation Accuracy: {accuracy * 100:.2f}%")

        # Collect true and predicted labels for confusion matrix
        all_true_labels.extend(y_val)
        all_pred_labels.extend(y_pred.flatten())

        accuracies.append(accuracy)
        fold += 1

    # Calculate and print the overall accuracy
    overall_accuracy = np.mean(accuracies)
    print(f"\nOverall Cross-Validation Accuracy: {overall_accuracy * 100:.2f}%")

    # Calculate and print the confusion matrix
    cm = confusion_matrix(all_true_labels, all_pred_labels)
    print("\nConfusion Matrix:")
    print(cm)

    print("Cross-validation completed.")

In [9]:
if __name__ == "__main__":
    main()

Processing Audio:   0%|          | 0/1191 [00:00<?, ?it/s]

Processing Audio: 100%|██████████| 1191/1191 [01:37<00:00, 12.24it/s]


Epoch 1/20
30/30 [==============================] - 8s 172ms/step - loss: 1.5395 - accuracy: 0.5809 - val_loss: 3.4855 - val_accuracy: 0.5816
Epoch 2/20
30/30 [==============================] - 5s 166ms/step - loss: 1.5416 - accuracy: 0.5924 - val_loss: 1.1495 - val_accuracy: 0.6904
Epoch 3/20
30/30 [==============================] - 5s 155ms/step - loss: 1.5149 - accuracy: 0.6040 - val_loss: 2.9216 - val_accuracy: 0.5858
Epoch 4/20
30/30 [==============================] - 4s 145ms/step - loss: 1.5856 - accuracy: 0.6061 - val_loss: 1.0348 - val_accuracy: 0.7531
Epoch 5/20
30/30 [==============================] - 5s 160ms/step - loss: 1.2135 - accuracy: 0.6071 - val_loss: 1.3279 - val_accuracy: 0.7238
Epoch 6/20
30/30 [==============================] - 5s 147ms/step - loss: 1.1289 - accuracy: 0.6071 - val_loss: 0.6831 - val_accuracy: 0.7531
Epoch 7/20
30/30 [==============================] - 4s 140ms/step - loss: 1.0529 - accuracy: 0.6187 - val_loss: 0.7747 - val_accuracy: 0.7238
Epoch 

/home/user/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 1s 33ms/step
Fold 1 - Validation Accuracy: 75.31%
Epoch 1/20
30/30 [==============================] - 7s 146ms/step - loss: 1.6005 - accuracy: 0.5971 - val_loss: 2.8667 - val_accuracy: 0.7269
Epoch 2/20
30/30 [==============================] - 4s 140ms/step - loss: 1.5473 - accuracy: 0.5971 - val_loss: 1.6229 - val_accuracy: 0.7395
Epoch 3/20
30/30 [==============================] - 4s 136ms/step - loss: 1.5614 - accuracy: 0.6139 - val_loss: 4.1140 - val_accuracy: 0.6345
Epoch 4/20
30/30 [==============================] - 4s 138ms/step - loss: 1.5679 - accuracy: 0.6097 - val_loss: 5.3960 - val_accuracy: 0.6345
Epoch 5/20
30/30 [==============================] - 4s 146ms/step - loss: 1.2762 - accuracy: 0.6128 - val_loss: 3.2267 - val_accuracy: 0.6555


...

8/8 [==============================] - 0s 34ms/step
Fold 3 - Validation Accuracy: 72.69%
Epoch 1/20
30/30 [==============================] - 7s 148ms/step - loss: 1.5220 - accuracy: 0.5918 - val_loss: 5.2993 - val_accuracy: 0.5546
Epoch 2/20
30/30 [==============================] - 4s 144ms/step - loss: 1.5320 - accuracy: 0.6212 - val_loss: 5.4591 - val_accuracy: 0.6092
Epoch 3/20
30/30 [==============================] - 4s 142ms/step - loss: 1.4674 - accuracy: 0.5950 - val_loss: 2.4987 - val_accuracy: 0.6513
Epoch 4/20
30/30 [==============================] - 4s 135ms/step - loss: 1.3621 - accuracy: 0.6317 - val_loss: 4.8007 - val_accuracy: 0.5084
Epoch 5/20
30/30 [==============================] - 4s 130ms/step - loss: 1.2362 - accuracy: 0.6338 - val_loss: 1.3735 - val_accuracy: 0.6639
Epoch 6/20
30/30 [==============================] - 4s 140ms/step - loss: 1.0762 - accuracy: 0.6527 - val_loss: 1.0473 - val_accuracy: 0.6723
Epoch 7/20
30/30 [==============================] - 4s 145m

/home/user/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 1s 34ms/step
Fold 4 - Validation Accuracy: 68.91%
Epoch 1/20
30/30 [==============================] - 7s 152ms/step - loss: 1.5179 - accuracy: 0.5782 - val_loss: 1.9774 - val_accuracy: 0.6218
Epoch 2/20
30/30 [==============================] - 4s 129ms/step - loss: 1.7887 - accuracy: 0.6159 - val_loss: 6.1542 - val_accuracy: 0.5252
Epoch 3/20
30/30 [==============================] - 4s 142ms/step - loss: 1.5794 - accuracy: 0.5981 - val_loss: 4.0036 - val_accuracy: 0.5084
Epoch 4/20
30/30 [==============================] - 5s 151ms/step - loss: 1.4116 - accuracy: 0.6275 - val_loss: 0.6360 - val_accuracy: 0.7437
Epoch 5/20
30/30 [==============================] - 4s 133ms/step - loss: 1.3091 - accuracy: 0.6118 - val_loss: 0.9196 - val_accuracy: 0.7059
Epoch 6/20
30/30 [==============================] - 4s 147ms/step - loss: 1.1277 - accuracy: 0.6422 - val_loss: 0.9219 - val_accuracy: 0.6218
Epoch 7/20
30/30 [==============================] - 4s 145m

/home/user/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 0s 30ms/step
Fold 5 - Validation Accuracy: 74.37%

Overall Cross-Validation Accuracy: 73.05%

Confusion Matrix:
[[509 126]
 [195 361]]
Cross-validation completed.
